In [6]:
import sys
try:
    from Optimizer import minimize
    from evaluate import *
except:
    from Implementation.Optimizer import minimize
    from Implementation.evaluate import *

try:
    sys.path.append('../AutoDiff')
    from variables import Variable
except:
    from AutoDiff.variables import Variable

    
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

## demo cases

In [2]:
f1=lambda x, y : 100*(y-x**2)**2 + (1-x)**2

In [3]:
v0_list = [[-1,1], [0,1], [2,1]]

In [4]:
models=['Conjugate Gradient','Steepest Descend','BFGS','Gradient Descend']

In [43]:
res={}
for model in models:
    res[model]=[minimize(f1,v0,method=model) for v0 in v0_list]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [5]:
res_BFGS=[minimize(f1,v0,method="BFGS") for v0 in v0_list]

In [38]:
import time
def _get_grad(fn, x, var_names):
    variables = [Variable(var_names[idx], x_n) for idx, x_n in enumerate(x)]
    out = fn(*variables)
    jacobian = out.jacobian()
    grad = np.array([jacobian[name] for name in var_names])
    return grad
def _line_search(fn, x, search_direction, grad, beta = 0.9, c = 0.9, alpha_init = 1):
    """approximately minimizes f along search_direction
    https://en.wikipedia.org/wiki/Backtracking_line_search
    """
    m = search_direction.T.dot(grad)
    alpha = alpha_init
    while (fn(*(x)) - fn(*(x+alpha*search_direction))) < -c*alpha*m:
        alpha = alpha * beta
    return alpha


class Result:
    def __init__(self, x, val_rec, time_rec, converge):
        """Record the optimization results and performance

        INPUTS
        =======
        x array: optimization value, can be either Variable or value. //Or just store the value, since there is no need for its derivatives
        val_rec array: stores the function inputs at each iteration, save for plotting the accuracy results.
        time_rec array: stores the cumulative time at each iteration.
        converge boolean: did the optimization procedure converge
        """
        self.x = x
        self.val_rec = val_rec
        self.time_rec = time_rec
        self.converge = converge
        # throw warning if not convergent


In [49]:
PRECISION = 1e-5
MAXITER = 1000
def BFGS(fn, x0, precision = PRECISION, max_iter = MAXITER, beta = 0.9, c = 0.9, alpha_init = 1, norm=np.inf):
    approx_hessian = np.eye(len(x0))
    
    x = np.array(x0, dtype=np.float)
    var_names = ['x'+str(idx) for idx in range(len(x))]
    
    val_rec = [x.copy()]
    time_rec = [0]
    init_time = time.time()
    
    
    for i in range(max_iter):
        grad_now = _get_grad(fn, x, var_names)
        s = np.linalg.solve(approx_hessian,-grad_now)
        x+=s
        val_rec.append(x.copy())
        
        # update matrix Hessian
        grad1=_get_grad(fn, x, var_names)
        y=grad1-grad_now
        dH1=np.outer(y,y)/np.dot(y,s)
        Hs=np.dot(approx_hessian,s)
        dH2=-np.outer(Hs,Hs)/np.dot(Hs,s)
        approx_hessian+=dH1+dH2
        
        time_rec.append(time.time()-init_time)
        
        if np.linalg.norm(grad1, norm) <= precision:
            # reshape val_rec
            val_rec = np.array(val_rec)
            time_rec = np.array(time_rec)
            return Result(x, val_rec, time_rec, True)
        
    converge = (np.linalg.norm(grad1, norm) <= precision)
    return Result(x, np.array(val_rec), time_rec, converge)

In [55]:
res=BFGS(f1,v0_list[2])

In [56]:
res.val_rec

array([[ 2.00000000e+00,  1.00000000e+00],
       [-2.40000000e+03,  6.01000000e+02],
       [ 1.86707487e+00,  6.39035624e+02],
       [ 3.00287671e+01, -1.34521175e+05],
       [ 1.99408493e+00,  1.47850670e+01],
       [ 1.99624515e+00,  4.16882903e+00],
       [ 1.99628252e+00,  3.98514569e+00],
       [ 1.99628252e+00,  3.98514594e+00],
       [ 1.99628227e+00,  3.98517986e+00],
       [ 1.99628177e+00,  3.98521304e+00],
       [ 1.99628002e+00,  3.98527620e+00],
       [ 1.99627555e+00,  3.98536365e+00],
       [ 1.99626316e+00,  3.98548960e+00],
       [ 1.99623048e+00,  3.98563982e+00],
       [ 1.99614365e+00,  3.98574921e+00],
       [ 1.99591517e+00,  3.98557360e+00],
       [ 1.99531420e+00,  3.98436666e+00],
       [ 1.99373618e+00,  3.97999697e+00],
       [ 1.98957835e+00,  3.96654390e+00],
       [ 1.97839965e+00,  3.92722034e+00],
       [ 1.94501629e+00,  3.80449334e+00],
       [ 1.72918306e+00,  2.99886740e+00],
       [ 2.28175930e+00,  5.05408726e+00],
       [ 1.